# This is the Clear Nature project notebook of Amir Grossman and Maya Attia from the Technion's ECE faculty.

Detectron2 + simple-lama-inpainting: \
Much more easy to run than previous version.\
at first we got good masks from detectron, but the inpainting was pretty bad, beside a few picters with small-area people.\
We added dilation to increase mask size to improve inpainting and to avoid small misses in mask generation.

In [ ]:
# Step 0: Install and setup dependencies
!pip install -q git+https://github.com/facebookresearch/detectron2.git opencv-python matplotlib gdown omegaconf einops pytorch_lightning
!pip install -q simple_lama_inpainting

!rm -rf /content/lama
!git clone -q https://github.com/advimman/lama.git /content/lama  # You can keep this if you want other lama utils


  Preparing metadata (setup.py) ... done


In [ ]:
# ⚡️ Step 1: Load Libraries
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

import torch
from PIL import Image

from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2 import model_zoo

from simple_lama_inpainting import SimpleLama

In [ ]:
# Step 2: Setup Detectron2
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 80
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")

# Set device dynamically
cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {cfg.MODEL.DEVICE}")

predictor = DefaultPredictor(cfg)


Using device: cpu


In [ ]:
# Step 3: Setup SimpleLama model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
lama_model = SimpleLama(device=device)

In [ ]:
# Step 4: Load images from /content/images
# image_dir = "/content/images"
image_dir = "/content/all_images"
os.makedirs(image_dir, exist_ok=True)
image_paths = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.webp'))]
image_paths = sorted(image_paths)

In [ ]:
# Step 5: Process images
for path in image_paths:
    image = cv2.imread(path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Detect people masks
    outputs = predictor(image_rgb)
    instances = outputs["instances"]
    person_mask = np.zeros(image.shape[:2], dtype=bool)
    for i, cls in enumerate(instances.pred_classes):
        if cls.item() == 0:  # class 0 is person
            mask = instances.pred_masks[i].cpu().numpy()
            person_mask = np.logical_or(person_mask, mask)

    person_mask_uint8 = (person_mask * 255).astype(np.uint8)
    kernel = np.ones((15, 15), np.uint8)  # change to image-size relative sizing?
    person_mask_dilated = cv2.dilate(person_mask_uint8, kernel, iterations=1)

    # Inpaint with SimpleLama
    inpainted = lama_model(image_rgb, person_mask_dilated)

    # Display results
    fig, axes = plt.subplots(1, 2, figsize=(12,6))
    axes[0].imshow(image_rgb)
    axes[0].set_title("Original")
    axes[0].axis('off')

    axes[1].imshow(inpainted)
    axes[1].set_title("People Removed")
    axes[1].axis('off')

    plt.show()

    plt.figure(figsize=(6, 6))
    plt.imshow(person_mask_dilated, cmap='gray')
    plt.title("Person Mask")
    plt.axis('off')
    plt.show()


Output hidden; open in https://colab.research.google.com to view.

TODO:
* Dynamic dilate kernel size (drawing form image size)
* Think about shadowing and abstractions
